In [2]:
from lib.reader import read_alleles_to_num
import numpy as np
import os

from sklearn.model_selection import LeaveOneOut

import argparse

from lib.config import *
import lib.row_cheker as rch

import time

In [62]:
import multiprocessing

In [21]:
from importlib import reload

In [72]:
rch = reload(rch)

In [9]:
np.random.seed(239)

parser = argparse.ArgumentParser()
parser.add_argument('-i', type=str, dest='input_table_path')
parser.add_argument('-o', type=str, dest='output_table_path')
parser.add_argument('-n', type=int, dest='rows_number', default=500)
args = parser.parse_args(["-i", "../data_as_npy.npy", "-o", "../res"])

In [10]:
args

Namespace(input_table_path='../data_as_npy.npy', output_table_path='../res', rows_number=500)

In [11]:
table = np.load(args.input_table_path)
print(table.shape)
case_control = np.array([0] * 407 + [1] * 21)

print("Starting on original table...")
table_2x3 = rch.check_initial_table(table, case_control.astype(bool))

assert all(table_2x3.sum(1).sum(1) == 428)

(428, 4382202)
Starting on original table...


In [12]:
np.save("../table_2x3", table_2x3)

In [13]:
table_2x3.shape

(4382202, 2, 3)

In [17]:
table = table.astype(int)

In [ ]:
loo = LeaveOneOut()

target_rows_list = []
target_pvals_list = []

chi2_hash = dict()

In [99]:
def do_all_for_single_split(index, table_2x3, case_control, table, chi2_hash, row_number):
    train_index, test_index = index[0], index[1]
    beg = time.time()
    rows, pvals = rch.check_loo_table(table_2x3,
                                      case_control[test_index],
                                      table[test_index].flatten(),
                                      chi2_hash,
                                      False,                                  
                                      cut=args.rows_number)
    end = time.time()
    #print("Done with index {} in {} seconds".format(test_index, end - beg))
    
    return rows, pvals, end - beg

In [59]:
for (train_index, test_index) in train_test_indexes[3:10]:
    beg = time.time()
    rows, pvals = rch.check_loo_table(table_2x3,
                                      case_control[test_index],
                                      table[test_index].flatten(),
                                      chi2_hash,
                                      cut=args.rows_number)
    
    
    
    target_rows_list.append(rows)
    target_pvals_list.append(pvals)
    end = time.time()
    print("Done with index {} in {} seconds".format(test_index, end - beg))
    
    np.save("../1_stage_results/rows/rows_" + str(test_index[0]), rows)
    np.save("../1_stage_results/pvals/pvals_" + str(test_index[0]), pvals)

  0%|          | 315/4382202 [00:00<23:12, 3146.18it/s]

Done with index [3] in 935.8729054927826 seconds


  0%|          | 318/4382202 [00:00<23:02, 3169.33it/s]

Done with index [4] in 972.4523255825043 seconds


  0%|          | 311/4382202 [00:00<23:31, 3104.87it/s]

...

Done with index [7] in 920.5783610343933 seconds


  0%|          | 308/4382202 [00:00<23:43, 3078.76it/s]

Done with index [8] in 902.1052443981171 seconds


100%|██████████| 4382202/4382202 [15:09<00:00, 4816.30it/s]


Done with index [9] in 910.5523960590363 seconds


In [ ]:
target_rows = np.concatenate([x.reshape(1,-1) for x in target_rows_list], 0)
target_pvals = np.concatenate([x.reshape(1,-1) for x in target_pvals_list], 0)

print(target_rows.shape)

np.save(args.output_table_path, target_rows)
np.save(args.output_table_path+"_pvals", target_pvals)

In [27]:
import pickle

In [60]:
len(chi2_hash)

319738

In [44]:
len(chi2_hash)

254387

In [61]:
pickle.dump(chi2_hash, open("../1_stage_results/chi2_hash_9", 'wb'))

In [53]:
np.save("../1_stage_results/rows/rows_1", target_rows_list[1])

In [54]:
np.save("../1_stage_results/pvals/pvals_1", target_pvals_list[1])

In [36]:
train_test_indexes = [[x,y] for x,y in loo.split(table)]

In [66]:
target_rows_list[0][:20]

array([1376664, 1270728,  992761, 2390826, 2273628, 3712756, 3592121,
       1874146, 2924633, 3592133, 2924632, 2924627, 4359920, 1470867,
       1665981, 2404856, 2404861, 4269047,   67442,  479780])

In [67]:
target_rows_list[3][:20]

array([1376664, 1270728,  992761, 2390826, 2273628, 3712756, 3592121,
       1874146, 3592133, 2924633, 2924632, 2924627, 4359920, 1470867,
       1665981, 2404856, 2404861, 4269047,   67442,  650711])

In [68]:
target_pvals_list[0][:20]

array([  3.89122540e-21,   9.29860702e-18,   4.74927819e-17,
         1.71168970e-15,   2.27848394e-14,   1.25258874e-13,
         1.35620811e-13,   1.87686407e-13,   1.91116997e-13,
         1.91313799e-13,   1.96361828e-13,   1.97937368e-13,
         2.06683279e-13,   1.30862014e-12,   2.46600718e-12,
         1.07457615e-11,   1.07457615e-11,   2.74591180e-11,
         3.07866232e-11,   7.28638373e-11])

In [69]:
target_pvals_list[3][:20]

array([  3.89122540e-21,   9.44807769e-18,   5.08217422e-17,
         1.71168970e-15,   2.27848394e-14,   1.25258874e-13,
         1.35620811e-13,   1.87686407e-13,   1.89215302e-13,
         1.92945486e-13,   1.97937368e-13,   1.99415799e-13,
         2.06230630e-13,   1.23771787e-12,   2.46600718e-12,
         1.07457615e-11,   1.07457615e-11,   2.74591180e-11,
         3.07866232e-11,   6.68154814e-11])

## test MP

In [158]:
pool = multiprocessing.Pool(6)

 46%|████▋     | 2036719/4382202 [06:05<07:01, 5568.29it/s]Process ForkPoolWorker-73:
Process ForkPoolWorker-74:
Process ForkPoolWorker-72:
Process ForkPoolWorker-75:
Process ForkPoolWorker-76:
Process ForkPoolWorker-77:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/anaconda/envs/py35/lib

  File "<ipython-input-99-47939cf8a5aa>", line 9, in do_all_for_single_split
    cut=args.rows_number)
  File "<ipython-input-99-47939cf8a5aa>", line 9, in do_all_for_single_split
    cut=args.rows_number)
  File "<ipython-input-99-47939cf8a5aa>", line 9, in do_all_for_single_split
    cut=args.rows_number)
  File "<ipython-input-149-b2edb26f4809>", line 2, in g
    return do_all_for_single_split(x, table_2x3, case_control, table, chi2_hash, args.rows_number)
  File "/home/arykachevskiy/gene_research/ASD_package/lib/row_cheker.py", line 103, in check_loo_table
    p_vals = check_p_vals(table, chi2_hash, update_hash, tests_mask)
  File "<ipython-input-99-47939cf8a5aa>", line 9, in do_all_for_single_split
    cut=args.rows_number)
  File "/home/arykachevskiy/gene_research/ASD_package/lib/row_cheker.py", line 103, in check_loo_table
    p_vals = check_p_vals(table, chi2_hash, update_hash, tests_mask)
  File "/home/arykachevskiy/gene_research/ASD_package/lib/row_cheker.py", line 103, in ch

In [159]:
def g(x):
    return do_all_for_single_split(x, table_2x3, case_control, table, chi2_hash, args.rows_number)

In [ ]:
answ_50_428 = pool.map(g, train_test_indexes[50:])

KeyboardInterrupt: 

Exception in thread Thread-43:
Traceback (most recent call last):
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/context.py", line 267, in _Popen
    return Popen(process_obj)
  File "/anaconda/envs/py35/lib/python3.5/multiprocessing/popen_fork.py", line 20, in __init__
    self._l

In [ ]:
answ_10_428

In [151]:
len(answ_0_10)

10

In [ ]:
for i, (r, p, t) in enumerate(answ_50_428):
    print(i + 50, t)
    np.save("../1_stage_results/rows/rows_" + str(i+50), r)
    np.save("../1_stage_results/pvals/pvals_" + str(i+50), p)

In [157]:
all(answ_0_10[2][0] == target_rows_list[3])

True

In [143]:
target_rows_list[2] == target_rows_list[3]

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,